In [13]:
from gensim.test.utils import datapath
from gensim.models import FastText
import pandas as pd
import nltk
from gensim.parsing.preprocessing import STOPWORDS
import re
import spacy
import numpy as np
import tqdm 
import pandas as pd
import torch
import os
torch.cuda.set_device(0)
nltk.download('stopwords') 
nltk.download('wordnet') 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\abdel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\abdel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [14]:
STOPWORDS

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'

In [16]:
if os.path.exists('D:/NLP_Fast_embedding/yelp_academic_dataset_tip.json'):
    data_tip = pd.read_json('yelp_academic_dataset_tip.json', lines=True)[:6000]


In [17]:
len(data_tip)

6000

In [18]:
tip_text = data_tip['text']

In [19]:
tip_text[:3]

0                       Avengers time with the ladies.
1    They have lots of good deserts and tasty cuban...
2               It's open even when you think it isn't
Name: text, dtype: object

In [23]:
! python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 262.6 kB/s eta 0:00:49
     --------------------------------------- 0.0/12.8 MB 262.6 kB/s eta 0:00:49
     --------------------------------------- 0.0/12.8 MB 245.8 kB/s eta 0:00:52
     --------------------------------------- 0.0/12.8 MB 245.8 kB/s eta 0:00:52
     --------------------------------------- 0.0/12.8 MB 245.8 kB/s eta 0:00:52
     --------------------------------------- 0.1/12.8 MB 204.8 kB/s eta 0:01:03
     --------------------------------------- 0.1/12.8 MB 204.8 kB/s eta 0:01:03
     --------------------------------------- 0.1/12.8 MB 187.3 kB/s eta 0:01:08


In [20]:
import spacy

def process_text(document,stopwords=STOPWORDS):
    nlp = spacy.load("en_core_web_sm")
    document = re.sub(r'\s+', ' ', document, flags=re.I) 
    document = re.sub(r'\W', ' ', str(document)) 
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document) # remove all single characters
    document = document.lower() 
    doc_spacy=nlp(document)
    lemma_txt = ["".join(token.lemma_) for token in doc_spacy]
    lemma_no_stop_txt = [word for word in lemma_txt if word not in STOPWORDS]
    lemma_no_stop_txt = [word for word in lemma_no_stop_txt if len(word) > 3]
    clean_txt = ' '.join(lemma_no_stop_txt)
    return lemma_no_stop_txt

tip_text_preprocessed = tip_text.apply(process_text)



OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [190]:
tip_text_preprocessed

0                     [avenger, time, lady]
1    [good, desert, tasty, cuban, sandwich]
2                             [open, think]
3                  [decent, fried, chicken]
4      [appetizer, platter, special, lunch]
Name: text, dtype: object

In [191]:
model3= FastText(vector_size=312,workers=20,window=3,sg=1, min_count=1,sentences=tip_text_preprocessed, epochs=1000)

In [247]:
model3.wv.vectors_vocab.shape 

(17, 312)

In [193]:
model3.ns_exponent #PROBABILITY OF NEGATIVE WORD SAMPLING

0.75

In [248]:
test_words="Pizza Burger Coffee Restaurant Delivery Review Menu Rating Service Ambience".split()
test_words=list(np.array(test_words).astype(str))

In [228]:
def test_words_andmodel_updating(model, test_words):
    similar_words = []
    un_similar_words=[]
    embedding_all_test_words_list_separable=[]
    for word in test_words:
        similar_word = model.wv.most_similar(positive=[word], topn=10)
        unsimilar_word =model.wv.most_similar(negative=[word], topn=10)
        embedding_each_test_word=model.wv[word]
        embedding_all_test_words_list_separable.append(embedding_each_test_word)
        similar_words.append(similar_word)
        un_similar_words.append(unsimilar_word)
        model3= FastText(vector_size=312,workers=20,window=3,sg=1, min_count=1,sentences=tip_text_preprocessed, epochs=1000)
    return pd.DataFrame({"test_embedding":embedding_all_test_words_list_separable,"most_similar":similar_words , "most_opposite":un_similar_words})


In [232]:
test_custom_trained_model_updating=test_words_andmodel_updating(model3,test_words)

In [253]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
test_custom_trained_model_updating.to_csv('/kaggle/working/my_trained_model_test__.csv', index=False)


In [249]:
test_custom_trained_model_updating

,test_embedding,most_similar,most_opposite
0,"[-0.00027251645, 0.000518687, 0.00017248245, -0.0003517396, -3.1526928e-05, 0.0004127542, 0.00033043357, 0.00017691078, -0.00033247782, 0.0008148456, -0.00015841676, -0.0010350355, -0.00096346706, -0.00093134784, 0.0003158565, -0.00037016653, 0.0007743644, -7.908499e-05, 0.00026119515, -0.00036939536, -0.0002929508, -0.00010356139, -0.00095394027, 0.00041542534, -0.00031758114, -9.925469e-05, 6.476715e-05, 0.00016385193, 0.00033464888, -3.4203993e-05, 0.00013819551, 0.0005093376, -0.00056486874, -0.0003087935, 6.603389e-07, 0.00061515433, 0.00038889382, 0.0005208441, -0.00043432004, 0.0004203203, 0.00031856354, -0.00039616218, 0.00018310311, 0.00015123338, 0.0002449364, -0.0003440882, -0.0004954358, 0.0015591856, -0.0005190933, -0.0012044924, 0.00011676656, 0.00022893595, -0.0005289714, 0.00036013013, -0.0011769601, 0.0003719489, 0.00019465433, -0.0004981683, 0.00014223666, -9.0237605e-05, 0.00022670467, -0.0004316125, -0.00053857313, -0.00047529754, -0.0004185599, -0.00079425547, 0.00047258573, 0.0005440756, 0.0006657563, -0.0002620446, 0.0011247195, 2.5652991e-05, -0.0003310391, -0.00034259076, 4.9481743e-05, 0.0006786558, 0.0007305237, 0.0005775327, 0.00046032056, -0.00053819816, 0.0004879276, -0.0001811775, 4.8070513e-05, -0.00021191107, 0.000760466, 0.0011416442, -0.00027041117, 0.00071012747, 0.0005420609, -0.0005574249, -0.0006893904, -0.00048048006, 4.8847105e-05, 8.197501e-06, -0.00042194786, 1.3169616e-05, -9.0972935e-05, -0.00058449106, -0.00020793953, 6.0625513e-05, ...]","[(lady, 0.11452455818653107), (fried, 0.10815078765153885), (lunch, 0.07257714867591858), (decent, 0.07186928391456604), (chicken, 0.06620976328849792), (tasty, 0.06206616014242172), (avenger, 0.04520556703209877), (platter, 0.0400925911962986), (desert, 0.03674844652414322), (sandwich, -0.002303817542269826)]","[(cuban, 0.11045144498348236), (time, 0.10858561098575592), (good, 0.08502629399299622), (special, 0.07010940462350845), (appetizer, 0.05047464743256569), (open, 0.03847501054406166), (think, 0.018723487854003906), (sandwich, 0.002303817542269826), (desert, -0.03674844652414322), (platter, -0.0400925911962986)]"
1,"[0.0003298372, -0.0004132004, -0.00061778614, 0.00045831347, -0.00047086334, 3.5996374e-05, -0.00021565048, 5.991583e-05, -0.00095733133, -0.0007473154, 0.000890351, -0.00029401135, 0.00011257693, -0.00012443814, 4.9213017e-05, 0.00059492124, -0.0006717219, 0.0005265188, 0.0006046901, 0.00026215872, -0.00033829047, -6.8421214e-05, -0.00054673455, 0.0007122989, 0.0001215974, -0.00036230448, 3.862384e-05, -0.00018644486, -0.00095512916, 0.00074636017, 0.00025995504, -0.0009722712, 0.0007729609, -0.00046362056, 0.0004657922, 0.00061387906, 0.00017303, 6.1441126e-05, -0.0005416803, -0.00018232771, -0.00058485486, -0.00032150184, -0.000313971, -0.0009779141, 0.0006222835, -0.0005439374, 0.00025279232, 0.00018387807, -0.0007847188, -0.000699125, -3.3888064e-05, -0.0009819745, 0.0002424845, 0.0003270374, -5.2937205e-05, 0.00012297345, -0.0004437351, 5.3460135e-05, -0.00020906908, 0.00012368693, 0.00091504015, -0.00010905117, 0.00024838053, -0.00070277415, -0.00083918736, -0.0003603259, 0.00061493635, -0.00022775546, 0.0006482092, 0.00021055443, -0.00085286016, 0.00034177935, -5.04732e-05, -8.8041386e-05, 0.0004520286, 0.0004173165, -8.849813e-05, -0.0006767212, -3.9258503e-05, -0.0003668465, 7.272639e-05, 2.7731077e-05, 3.0263265e-05, -0.00021611893, -0.00059504644, 0.00038293077, -0.0003921547, 0.0003887229, 0.0004918802, 0.00066559896, -0.000788384, -4.7805494e-05, -0.0004470669, 0.00028839716, -0.0004662188, -0.0009110603, 0.00021638122, -8.4124484e-05, -0.00024388309, 0.0003810703, ...]","[(avenger, 0.2105431705713272), (sandwich, 0.1668359786272049), (chicken, 0.07776130735874176), (special, 0.06825065612792969), (open, 0.053561653941869736), (desert, 0.046331025660037994), (fried, 0.04477228969335556), (lunch, 0.03665320575237274), (appetizer, 0.03206630051136017), (time, 0.020937332

In [246]:
np.array(test_custom_trained_model_updating['test_embedding'][0]).shape

(312,)

# Facebook pretrained model

In [264]:
!pip install fasttext

In [276]:
!export KAGGLE_CONFIG_DIR=/root/.kaggle
# !mkdir ~/.kaggle
# !cp /root/.kaggle/kaggle.json ~/.kaggle/kaggle.json

!kaggle datasets download -d fasttext/crawl-300d-2m -p /kaggle/working


Traceback (most recent call last):
  File "/opt/conda/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/opt/conda/lib/python3.10/site-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/opt/conda/lib/python3.10/site-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [283]:
from gensim.models.fasttext import load_facebook_model

# download link https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz

model_PRETRAINED= load_facebook_model('cc.ar.300.bin')

# Now you can use the model
similar_words = model.wv.most_similar(positive=['word'], topn=5)
print(similar_words)


FileNotFoundError: [Errno 2] No such file or directory: 'cc.ar.300.bin'

In [284]:
def test_model_basedon_facebook_pretrained_model(model_1,model_2,test_words):
    check_list=[]
    for word in test_words:
        check_list.append(np.allclose(model_1.wv[word],model_2.wv[word]))
    return check_list


In [285]:
test_model_basedon_facebook_pretrained_model(model_PRETRAINED,model3,test_words)

NameError: name 'model_PRETRAINED' is not defined